# SPECT Ordered Subsets Expectation Maximisation Notebook
A notebook to demonstrate the setup and basic OSEM reconstruction of a 2-dimensional dummy image using SIRF's SPECT projector

# What is OSEM?
The following is just a very brief explanation of the concepts behind OSEM.

PET reconstruction is commonly based on the *Maximum Likelihood Estimation (MLE)* principle. The *likelihood* is the probability to observe some measured data given a (known) image. MLE attempts to find the image that maximises this likelihood. This needs to be done iteratively as the system of equations is very non-linear.

A common iterative method uses *Expectation Maximisation*, which we will not explain here. The resulting algorithm is called *MLEM* (or sometimes *EMML*). However, it is rather slow. The most popular method to increase computation speed is to compute every image update based on only a subset of the data. Subsets are nearly always chosen in terms of the "views" (or azimuthal angles). The *Ordered Subsets Expectation Maximisation (OSEM)* cycles through the subsets. More on this in another notebook, but here we just show how to use the SIRF implementation of OSEM.

OSEM is (still) the most common algorithm in use in clinical PET.

# Initial Setup

In [ ]:
import os
import glob
import numpy as np

import notebook_setup


from sirf.STIR import show_2D_array
import sirf.STIR as spect
from sirf.Utilities import examples_data_path
from sirf_exercises import cd_to_working_dir

data_path = examples_data_path('SPECT')

cd_to_working_dir("SPECT", "OSEM")

Redirect information, warning and error messages to log files

In [ ]:
msg_red = spect.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')

Some useful function definitions

In [ ]:
def create_sample_image(image):
    '''fill the image with some simple geometric shapes.'''
    image.fill(0)
    # create a shape
    shape = spect.EllipticCylinder()
    shape.set_length(400)
    shape.set_radii((100, 40))
    shape.set_origin((0, 60, 10))

    # add the shape to the image
    image.add_shape(shape, scale = 1)

    # add another shape
    shape.set_radii((30, 30))
    shape.set_origin((60, -30, 10))
    image.add_shape(shape, scale = 1.5)

    # add another shape
    shape.set_origin((-60, -30, 10))
    image.add_shape(shape, scale = 0.75)

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV."""
    cyl_filter =spect.TruncateToCylinderProcessor()
    cyl_filter.apply(image)
    return image

def add_noise(proj_data,noise_factor = 1):
    """Add Poission noise to acquisition data."""
    proj_data_arr = proj_data.as_array() / noise_factor
    # Data should be >=0 anyway, but add abs just to be safe
    proj_data_arr = np.abs(proj_data_arr)
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.clone()
    noisy_proj_data.fill(noisy_proj_data_arr);
    return noisy_proj_data

# Create ground truth and simulated data images

First, we create a template acquisition data object from file

In [ ]:
templ_sino = spect.AcquisitionData(os.path.join(data_path,'template_sinogram.hs'))

We then use this template sinogram to create simple ground truth and attenuation images

In [ ]:
# create ground truth image
image = templ_sino.create_uniform_image()
create_sample_image(image)
image = image.zoom_image(zooms=(0.5, 1.0, 1.0)) #required for now because SPECT is 360 degree acquisiton

# create attenuation image
uMap = templ_sino.create_uniform_image()
create_sample_image(uMap)
uMap = uMap.zoom_image(zooms=(0.5, 1.0, 1.0))*0.1

In [ ]:
# show the ground truth image
image_array = image.as_array()
show_2D_array('Phantom image', image_array[0,:,:])

# show the attenuation image
uMap_array = uMap.as_array()
show_2D_array('Attenuation image', uMap_array[0,:,:])

We then create the acquisition model by first creating an acquisition matrix (SPECTUBMatrix) object, incluidng models of attenuationa and resolution and apply this to a sirf Acquisition Model (AcquisitionModelUsingMatrix) object

In [ ]:
# select acquisition model that implements the geometric
# forward projection by a ray tracing matrix multiplication
acq_model_matrix = spect.SPECTUBMatrix();

### here we add attenuation and set a PSF for the aquisition model ###
acq_model_matrix.set_attenuation_image(uMap) # add attenuation
acq_model_matrix.set_resolution_model(0.2,0.2,full_3D=False) #resolution modelling

acq_model = spect.AcquisitionModelUsingMatrix(acq_model_matrix)
acq_model.set_up(templ_sino, image)

Next, forward project our ground truth image and add noise to simulate noisy acquired data.

In [ ]:
print('projecting image...')
# project the image to obtain simulated acquisition data
# data from raw_data_file is used as a template
acq_model.set_up(templ_sino, image)
simulated_data = templ_sino.get_uniform_copy()
acq_model.forward(image, 0, 1, simulated_data)

  # create noisy data
noisy_data = simulated_data.clone()
noisy_data_as_array = np.random.poisson(simulated_data.as_array())
noisy_data.fill(noisy_data_as_array)

# show simulated acquisition data
simulated_data_as_array = simulated_data.as_array()
show_2D_array('Forward projection', simulated_data_as_array[0, 0,:,:])
show_2D_array('Forward projection with added noise', noisy_data_as_array[0, 0,:,:])

In [ ]:
# create objective function
obj_fun = spect.make_Poisson_loglikelihood(noisy_data)

### and now we apply a different resolution mdoel ### and
acq_model_matrix.set_resolution_model(0.1,0.1,full_3D=False)
acq_model_2 = spect.AcquisitionModelUsingMatrix(acq_model_matrix)
obj_fun.set_acquisition_model(acq_model_2)

# create OSEM reconstructor object
num_subsets = 21 # number of subsets for OSEM reconstruction
num_subiters = 42 #number of subiterations (i.e two full iterations)
OSEM_reconstructor = spect.OSMAPOSLReconstructor()
OSEM_reconstructor.set_objective_function(obj_fun)
OSEM_reconstructor.set_num_subsets(num_subsets)
OSEM_reconstructor.set_num_subiterations(num_subiters)

In [ ]:
# create initialisation image and set up reconstructor
init_image = make_cylindrical_FOV(image.get_uniform_copy(1))
OSEM_reconstructor.set_up(init_image)

In [ ]:
# Reconstruct and show reconstructed image
OSEM_reconstructor.reconstruct(init_image)
out_image = OSEM_reconstructor.get_current_estimate()
out_image_array = out_image.as_array()
show_2D_array('Reconstructed image', out_image_array[0,:,:])

What is different about this image?

In [ ]:
#%% delete temporary files
wdpath = os.getcwd()
for filename in glob.glob(os.path.join(wdpath, "tmp*")):
    os.remove(filename) 